In [1]:
# imports
import os
import json
import re

import torch 
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import numpy as np

from MDP import MDP

import stable_baselines3
import sb3_contrib

import gym

In [2]:
# check torch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3080'

In [3]:
# create Neural Network

class Net(nn.Module):
    """
    input : 2 X 4 X 4 grid
    label : Move [0;6]
    """
    def __init__(self):
        super(Net, self).__init__()
        # first layer: input
        self.conv1 = nn.Conv2d(2, 32, 2, padding = 1)

        #second layer : 2nd convolution
        self.conv2 = nn.Conv2d(32, 64, 3, padding= 1)

        self.conv3 = nn.Conv2d(64, 128, 2)

        self.conv4 = nn.Conv2d(128, 64, 3)

        self.fc1 = nn.Linear(64, 32)

        self.fc2 = nn.Linear(32, 16)

        self.out = nn.Linear(16, 6)


    def forward(self, x):
        x = x.float()

        x = F.relu(self.conv1(x))

        x = F.relu(self.conv2(x))

        x = F.relu(self.conv3(x))

        x = F.relu(self.conv4(x))

        x = F.max_pool2d(x, kernel_size = 2, stride = 2)

        x = torch.flatten(x,start_dim=1)

        x = F.relu(self.fc1(x))

        x = F.relu(self.fc2(x))

        x = self.out(x)
    
        return x      

In [4]:
#creating model
net = Net()
net.cuda()
print(net)

params = list(net.parameters())
print(f"number of parameters: {len(params)}")

#loss function
loss = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.Adam(net.parameters())
optimizer

Net(
  (conv1): Conv2d(2, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
  (conv4): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (out): Linear(in_features=16, out_features=6, bias=True)
)
number of parameters: 14


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [5]:
net.load_state_dict(torch.load("Net"))

<All keys matched successfully>

In [64]:
#custom environment
from gym import spaces

class Gridworld(gym.Env):

    metadata = {"render.modes" : ["human"]}

    def __init__(self, episodes, dir = ["data_easy", "generated_easy", "data_medium", "generated_med", "data", "generated_imitation"], type = "train", p = [0.15, 0.2, 0.15, 0.15, 0.3, 0.15], lambda1 = 0.01, lambda2 = 0.1, lambda3 = 1, load_optimal = False, epsilon = 0.1) -> None:
        super(Gridworld, self).__init__()
        self.action_space = spaces.Discrete(6)
        self.observation_space = spaces.Box(low = 0, high = 10, shape = (2, 4, 4))

        #available MDPs
        self.dir = dir
        self.type = type
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.lambda3 = lambda3

        self.actions = ["move", "turnLeft", "turnRight", "pickMarker", "putMarker","finish"]
        self.actionsDict = {"move" : 0, "turnLeft" : 1, "turnRight" : 2, "pickMarker" : 3, "putMarker" : 4, "finish" : 5}

        self.epsilon = epsilon

        self.load_optimal = load_optimal

        #generate curriculum
        #{"data_easy" : 15% ,"generated_easy" : 20% , "data_medium" : 15% , "generated_med" : 15% "data" : 30%, "generated_imitation" : 15%}
        self.curriculum = []
        #data_easy
        for directory,amount in zip(dir, p):
            tasks = list(np.random.choice(os.listdir(os.sep.join(["datasets", directory, type, "task"])), int(episodes * amount)))
            tasks = zip([directory for _ in range(int(episodes * amount))], tasks)
            self.curriculum += tasks
        self.currentTask = 0

    def reset(self):
        nextDir, self.nexti = self.curriculum[self.currentTask]
        self.nexti = re.sub(r"\D", "", self.nexti)
        self.currentTask += 1
        self.currentMDP = MDP(nextDir, self.type, self.nexti, lambda1= self.lambda1, lambda2 = self.lambda2, lambda3 =self.lambda3)

        # load optimal sequence if possible
        try:
            with open(os.sep.join(["datasets", nextDir, self.type, "seq", self.nexti + "_seq.json"])) as grid:
                grid = json.load(grid)
                self.optimal_seq = grid["sequence"]
                # current step
                assert self.optimal_seq[-1] == "finish"
        except:  
            self.optimal_seq = []
        self.steps = 0

        #with probability 1 - epsilon use either the optimal sequence or better masks
        self.use_optimal =  (np.random.rand() < (1 - self.epsilon)) and self.load_optimal

        return self.currentMDP.get_current_state()/10

    def step(self, action):
        nextState, rew, done, info = self.currentMDP.sample_next_state_and_reward(self.actions[action])
        self.steps += 1
        if self.steps > 500:
            return nextState, -self.lambda3, True, info 

        return nextState/10, rew - 0.01, done, info 
        
    def render(self):
        self.currentMDP.print_grid()

    def close(self):
        pass
    
    def action_masks(self):

        mat = self.currentMDP.get_current_state()
        if np.array_equal(mat[0], mat[1]):
            return np.array([0,0,0,0,0,1])

        # force agent to take optimal action if possible
        mask = self.currentMDP.action_mask()

        if self.use_optimal and self.optimal_seq:
            mask = np.zeros(6)
            mask[self.actionsDict[self.optimal_seq[self.steps]]] = 1
            return mask
        
        return mask

    # functions bellow are only used for inheritance 
    def get_MDP(self):
        return self.currentMDP

    def get_MDP_name(self):
        return self.nextDir, self.nextType, self.nexti

In [37]:
testEnv = Gridworld(100, dir = ["generated_med"], load_optimal= True)

In [42]:
testEnv.reset()
testEnv.render()
print(testEnv.action_masks())
_, r, _, _ = testEnv.step(np.argmax(testEnv.action_masks()))
print(r)
testEnv.render()
testEnv.action_masks()
_, r, _, _ = testEnv.step(np.argmax(testEnv.action_masks()))
print(r)
testEnv.render()

[[['#' '#' '.' '.']
  ['.' '.' '.' '.']
  ['v' '.' '#' '#']
  ['.' '.' '.' '#']]

 [['#' '#' '.' '.']
  ['.' '.' '.' '.']
  ['.' '.' '#' '#']
  ['v' '.' '.' '#']]]
[1. 1. 1. 0. 0. 0.]
0.0
[[['#' '#' '.' '.']
  ['.' '.' '.' '.']
  ['.' '.' '#' '#']
  ['v' '.' '.' '#']]

 [['#' '#' '.' '.']
  ['.' '.' '.' '.']
  ['.' '.' '#' '#']
  ['v' '.' '.' '#']]]
0.99
[[['#' '#' '.' '.']
  ['.' '.' '.' '.']
  ['.' '.' '#' '#']
  ['v' '.' '.' '#']]

 [['#' '#' '.' '.']
  ['.' '.' '.' '.']
  ['.' '.' '#' '#']
  ['v' '.' '.' '#']]]


In [61]:
np.random.seed(123)

def test_RL_models(model):
    totalCorrect, totalOptimal = 0,0
    for dir, num in zip(["data_easy", "data_medium", "data"], [80, 24, 480]):
        print("current data: " + dir)
        total_steps = 0
        valDataset = Gridworld(num * 10, dir = [dir], p = [1], type = "val", lambda1=0, lambda2=0, load_optimal= False)
        correct, total, optimal = 0, num*10, 0
        for task in range(int(total)):
            if task % num == num-1:
                print(f"{(task+1) / total *100} %, running acc: {(correct*100)/(task+1)}, task solved optimaly: {optimal*100/task} %, average steps to solve: {total_steps / task}")
            currMDP = valDataset.reset()
            lenOptimalSeq = len(valDataset.optimal_seq)
            done = False
            steps = 0
            while not done and steps < 50:
                action = model.predict(currMDP, action_masks = valDataset.action_masks())[0]
                currMDP, rew, done, _ = valDataset.step(action)
                steps += 1
                if rew > 0:
                    correct += 1
                    total_steps += steps
                    if steps == lenOptimalSeq:
                        optimal += 1

                
        

In [55]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.env_util import make_vec_env

import torch.nn.functional as F
from torch import nn

class CustomFeatureExtractorTorch(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space, features_dim: int = 16):
        super().__init__(observation_space, features_dim)

        self.conv1 = net.conv1
        self.conv2 = net.conv2
        self.conv3 = net.conv3
        self.conv4 = net.conv4
        
        self.fc1 = net.fc1
        self.fc2 = net.fc2
  
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))

        x = F.max_pool2d(x, kernel_size = 2, stride = 2)
        x = torch.flatten(x, start_dim=1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        return x

net_arch = [
    64, 128,
    dict(vf = [64, 32, 18], pi = [32, 16, 6])
]

policy_kwargs = dict(
    features_extractor_class = CustomFeatureExtractorTorch,
    net_arch = net_arch
)

In [11]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.env_util import make_vec_env
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy

In [12]:
policy_kwargs = dict(
    features_extractor_class = CustomFeatureExtractorTorch,
    net_arch = net_arch
)

In [66]:
episodes = 1e6

FinalEnv = make_vec_env(Gridworld, n_envs= 2  , 
    env_kwargs={"episodes" : episodes, "lambda1" : 0.1, "lambda2" : 0.5, "lambda3": 1, 
            "dir" : ["data_easy", "data_medium", "data", "generated_imitation", "generated_easy", "generated_med"], "load_optimal" : True, "epsilon" : 0.5})

FinalModel = MaskablePPO(MaskableActorCriticPolicy, FinalEnv, policy_kwargs = policy_kwargs verbose = 1, n_steps= 500)

FinalModel.learn(episodes)

Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12.7     |
|    ep_rew_mean     | -0.0894  |
| time/              |          |
|    fps             | 1394     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 1000     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.09        |
|    ep_rew_mean          | 0.508       |
| time/                   |             |
|    fps                  | 976         |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 2000        |
| train/                  |             |
|    approx_kl            | 0.008106301 |
|    clip_fraction        | 0.0533      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.788      |
|    explained_variance   | -0.0346     |
|    learnin

In [248]:
FinalModel.learn(1.5 * 1e5)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 13.9     |
|    ep_rew_mean     | 1.14     |
| time/              |          |
|    fps             | 137      |
|    iterations      | 1        |
|    time_elapsed    | 14       |
|    total_timesteps | 2000     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 23.5        |
|    ep_rew_mean          | 1.02        |
| time/                   |             |
|    fps                  | 174         |
|    iterations           | 2           |
|    time_elapsed         | 22          |
|    total_timesteps      | 4000        |
| train/                  |             |
|    approx_kl            | 0.038805638 |
|    clip_fraction        | 0.176       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.415      |
|    explained_variance   | 0.504       |
|    learning_rate        | 0.

In [67]:
test_RL_models(FinalModel)

current data: data_easy
10.0 %, running acc: 83.75, task solved optimaly: 49.36708860759494 %, average steps to solve: 7.037974683544304
20.0 %, running acc: 86.25, task solved optimaly: 47.16981132075472 %, average steps to solve: 8.28930817610063
30.0 %, running acc: 87.08333333333333, task solved optimaly: 49.37238493723849 %, average steps to solve: 7.364016736401673
40.0 %, running acc: 87.1875, task solved optimaly: 48.90282131661442 %, average steps to solve: 7.545454545454546
50.0 %, running acc: 88.75, task solved optimaly: 52.13032581453634 %, average steps to solve: 7.280701754385965
60.0 %, running acc: 88.125, task solved optimaly: 51.774530271398746 %, average steps to solve: 6.8684759916492695
70.0 %, running acc: 88.03571428571429, task solved optimaly: 52.05724508050089 %, average steps to solve: 6.722719141323792
80.0 %, running acc: 88.125, task solved optimaly: 51.95618153364632 %, average steps to solve: 6.657276995305164
90.0 %, running acc: 88.05555555555556, tas

In [16]:
testEnv = Gridworld(load_optimal= False)

In [62]:
curr = testEnv.reset()
testEnv.render()

[[['.' '.' '.' '.']
  ['.' '.' '#' '#']
  ['.' '^' '.' '#']
  ['.' '#' '#' '#']]

 [['.' '.' '.' '.']
  ['.' '^' '#' '#']
  ['.' '.' '.' '#']
  ['.' '#' '#' '#']]]


In [61]:
a = FinalModel.predict(curr, action_masks= testEnv.action_masks(), deterministic= False)[0]
curr, _, done, _ = testEnv.step(a)
print(testEnv.action_masks())
testEnv.render()
if done:
    print("finished")

[0 0 0 0 0 1]
[[['.' '#' '#' '.']
  ['.' '.' '.' '.']
  ['.' '.' 'O' 'd']
  ['#' '.' '.' '.']]

 [['.' '#' '#' '.']
  ['.' '.' '.' '.']
  ['.' '.' 'O' 'd']
  ['#' '.' '.' '.']]]
